In [1]:
import pandas as pd 
import numpy as np 
import os
import cv2
import matplotlib.pyplot as plt
import warnings
import tensorflow 
import scipy

from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, Input, AveragePooling2D,Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow import keras
import tensorflow as tf

In [2]:
train_path = './dataset/train'

In [3]:
batch_size = 8
img_height = 229
img_width = 229

In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_path,
  validation_split=0.2,
  subset="training",
  seed=123,
  label_mode='categorical',
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 5121 files belonging to 4 classes.
Using 4097 files for training.


In [5]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  train_path,
  validation_split=0.2,
  subset="validation",
  seed=123,
  label_mode='categorical',
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 5121 files belonging to 4 classes.
Using 1024 files for validation.


In [6]:
class_names = train_ds.class_names
print(class_names)

['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']


In [7]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
vgg = VGG19(input_shape=(img_height,img_width,3), weights='imagenet', include_top=False)
for layer in vgg.layers:
    layer.trainable = False
x = Flatten()(vgg.output)

prediction = Dense(4, activation='softmax')(x)

modelvgg = Model(inputs=vgg.input, outputs=prediction)
modelvgg.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 229, 229, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 229, 229, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 229, 229, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 114, 114, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 114, 114, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 114, 114, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 57, 57, 128)       0     

In [9]:
modelvgg.compile(optimizer='adam',
loss='categorical_crossentropy',
metrics=['acc'])

In [10]:
modelvgg.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15
513/513 [==============================] - 184s 321ms/step - loss: 8.6320 - acc: 0.6161 - val_loss: 8.5006 - val_acc: 0.6494
Epoch 2/15
513/513 [==============================] - 158s 308ms/step - loss: 3.4812 - acc: 0.8103 - val_loss: 5.8641 - val_acc: 0.7100
Epoch 3/15
513/513 [==============================] - 163s 318ms/step - loss: 2.3602 - acc: 0.8719 - val_loss: 4.5643 - val_acc: 0.7539
Epoch 4/15
513/513 [==============================] - 164s 321ms/step - loss: 1.5600 - acc: 0.8985 - val_loss: 4.4621 - val_acc: 0.8076
Epoch 5/15
513/513 [==============================] - 168s 327ms/step - loss: 2.5459 - acc: 0.8848 - val_loss: 17.5781 - val_acc: 0.6484
Epoch 6/15
513/513 [==============================] - 165s 322ms/step - loss: 3.3226 - acc: 0.8760 - val_loss: 4.1020 - val_acc: 0.8496
Epoch 7/15
513/513 [==============================] - 170s 331ms/step - loss: 1.4792 - acc: 0.9295 - val_loss: 4.8433 - val_acc: 0.8438
Epoch 8/15
513/513 [===========================

In [11]:
loss, accuracy = modelvgg.evaluate(val_ds)
print("Accuracy: ", accuracy)

128/128 [==============================] - 30s 236ms/step - loss: 5.0469 - acc: 0.8887
Accuracy:  0.888671875
